In [ ]:
import cv2
import os
import tensorflow as tf
from tensorflow.keras.models import load_model
from pygame import mixer
import time
import numpy as np

In [ ]:
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye.xml")
model = tf.keras.models.load_model(r'C:\Users\imadc\Super Folder\modelss\MobileNet.h5')

In [ ]:
def preprocess_eye(eye):
    eye = cv2.resize(eye, (80, 80))
    eye = eye / 255.0
    eye = np.expand_dims(eye, axis=0)
    return eye

In [ ]:
from pygame import mixer

cap = cv2.VideoCapture(0)
song = r"C:\Users\imadc\alert.wav"
mixer.init()
alert_sound = mixer.Sound(song)
point= 0
prev_frame_time = 0
sound_flag = False

#AirBag Model 

from tensorflow.keras.preprocessing import image
airbagmodel = tf.keras.models.load_model(r'C:\Users\imadc\Super Folder\modelss\MobileNetAirBagModel.h5')
alert_threshold=10
consecutive_frames = 0
alert_triggered = False


closed_eye_frames = 0
continuous_closed_eye_frames = 0


#Yawn

import dlib
import math

predictor_path = r"C:\Users\imadc\Super Folder\shape_predictor_68_face_landmarks.dat"
predictor = dlib.shape_predictor(predictor_path)
face_detector = dlib.get_frontal_face_detector()

yawn = r"C:\Users\imadc\yawn_alert.mp3"
yawn_alert = mixer.Sound(yawn)

consecutive_frames_yawning = 0
yawning_threshold = 8 
yawn_detected = False  
yawn_count = 0
yawn_alert_threshold = 4 
alert_displayed = False




